In [1]:
%reload_ext autoreload
%autoreload 2

import gymnasium as gym
import numpy as np
import itertools as iter
import random
from IPython import display
from PIL import Image

env = gym.make('FrozenLake-v1', render_mode='rgb_array')


class MC_episode():
        def __init__(self) -> None:
                self.G = 0
                self.W = 1
                self.Q = np.zeros([16, 4])
                self.C = np.zeros([16, 4])
                self.s_a_in_episode = []
                self.rewards_in_episode = []
                self.pi = {}
                for s in range(16):
                        self.pi.update({s:[0,0]}) # s:a,q, initialize target policy (deterministic)

                self.state = 0
                self.action = 0     

                self.iteration = 0
                

episodes = 5000
GAMMA = 1.0
mc = MC_episode()
 
# run episodes
env.reset()
env.render()

screen = env.render()
images = [Image.fromarray(screen)]

# run one episode
while mc.iteration<=episodes:
    mc.state=env.reset()
    done = 0
    mc.s_a_in_episode.clear()
    mc.rewards_in_episode.clear()
    while not done:
        if type(mc.state) != int:
              mc.state = mc.state[0]
        mc.s_a_in_episode.append((mc.state, mc.action))

        mc.action = env.action_space.sample() # behavior policy: random policy
        mc.state, reward, done, _,_ = env.step(mc.action) 
        mc.rewards_in_episode.append(reward)
    mc.G=0
    mc.W=1
    if mc.W != 0:
        for t in range(len(mc.s_a_in_episode)-1,-1,-1):
                mc.G = GAMMA*mc.G + mc.rewards_in_episode[t]
                s_a = mc.s_a_in_episode[t]
                mc.C[s_a[0],s_a[1]] = mc.C[s_a[0],s_a[1]] + mc.W
                mc.Q[s_a[0],s_a[1]] = mc.Q[s_a[0],s_a[1]] + mc.W/mc.C[s_a[0],s_a[1]] * (mc.G-mc.Q[s_a[0],s_a[1]])
                mc.pi[s_a[0]] = np.argmax(mc.Q[s_a[0],:])
                if s_a[1] != mc.pi[s_a[0]]:break
                mc.W = mc.W * (1/0.25)


    mc.iteration+=1
    # rendering
    screen = env.render()
    images.append(Image.fromarray(screen))



# Save GIF image
image_file = 'frozenlake-v1.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(
    image_file, save_all=True, append_images=images[1:], loop=0, duration=1)

# # The game is finished, so close the window
env.close()
print(mc.Q)
print(mc.pi)



[[0.9999938  0.99866283 0.99999978 0.        ]
 [0.         0.99959962 0.99998854 0.        ]
 [0.         0.25806452 0.99997044 0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.99719735 0.99579867 0.99272904]
 [0.         0.         0.         0.        ]
 [0.         0.         0.47222222 0.        ]
 [0.         0.         0.         0.        ]
 [0.97326839 0.         0.92055765 0.88352028]
 [0.         0.         0.96548486 0.67830424]
 [0.05970149 0.06060606 0.33928571 0.96      ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.64       0.        ]
 [0.86666667 0.5        0.53846154 0.63636364]
 [0.         0.         0.         0.        ]]
{0: 2, 1: 2, 2: 2, 3: 0, 4: 1, 5: [0, 0], 6: 2, 7: [0, 0], 8: 0, 9: 2, 10: 3, 11: [0, 0], 12: [0, 0], 13: 2, 14: 0, 15: [0, 0]}
